<a href="https://colab.research.google.com/github/kor1999/Sentence-Similarity-LLM-course-/blob/main/Sentence_similarity_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [19]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation, util
from torch.utils.data import DataLoader

In [ ]:
# Loading dataset of STS Benchmark
dataset = load_dataset('stsb_multi_mt', 'en')

# print info about dataset
print(dataset)

# Dividing dataset to train, validation and test
train_data = dataset['train']
val_data = dataset['dev']
test_data = dataset['test']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1379 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1500 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'similarity_score'],
        num_rows: 5749
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'similarity_score'],
        num_rows: 1379
    })
    dev: Dataset({
        features: ['sentence1', 'sentence2', 'similarity_score'],
        num_rows: 1500
    })
})


In [ ]:
# print examples of train data
print(train_data[:2])

{'sentence1': ['A plane is taking off.', 'A man is playing a large flute.'], 'sentence2': ['An air plane is taking off.', 'A man is playing a flute.'], 'similarity_score': [5.0, 3.799999952316284]}


In [ ]:
# Преобразование данных в InputExample
train_examples = [InputExample(texts=[row['sentence1'], row['sentence2']], label=row['similarity_score']) for row in train_data]
val_examples = [InputExample(texts=[row['sentence1'], row['sentence2']], label=row['similarity_score']) for row in val_data]
test_examples = [InputExample(texts=[row['sentence1'], row['sentence2']], label=row['similarity_score']) for row in test_data]


In [ ]:
# Создание DataLoader
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
val_dataloader = DataLoader(val_examples, shuffle=False, batch_size=16)
test_dataloader = DataLoader(test_examples, shuffle=False, batch_size=16)

In [ ]:
# Создание DataLoader
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
val_dataloader = DataLoader(val_examples, shuffle=False, batch_size=16)
test_dataloader = DataLoader(test_examples, shuffle=False, batch_size=16)

# Загрузка предобученной модели
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
# Настройка функции потерь
train_loss = losses.CosineSimilarityLoss(model)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Обучение модели
model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=4,
          warmup_steps=100,
          evaluator=None,
          evaluation_steps=0,
          output_path=None)


Step,Training Loss


In [ ]:
# Оценка модели на валидационных данных
evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(val_examples, name='sts-dev')
results = model.evaluate(evaluator)
print("Pearson correlation (cosine similarity):", results['sts-dev_pearson_cosine'])

#model.evaluate(val_dataloader)


In [20]:
# Функция для вычисления косинусного сходства между двумя словами
def check_similarity(word1, word2, model):
    # Получение эмбеддингов для двух слов
    embedding1 = model.encode(word1, convert_to_tensor=True)
    embedding2 = model.encode(word2, convert_to_tensor=True)

    # Вычисление косинусного сходства
    cosine_similarity = util.pytorch_cos_sim(embedding1, embedding2)

    return cosine_similarity.item()

In [32]:
# Пример слов для проверки

word1 = "cat"
word2 = "dog"
word3 = "table"
word4 = "tank"
word5 = "cats"
# Вычисление и вывод схожести

print(f"Cosine similarity between '{word1}' and '{word2}': {check_similarity(word1, word2, model)}")
print(f"Cosine similarity between '{word1}' and '{word3}': {check_similarity(word1, word3, model)}")
print(f"Cosine similarity between '{word1}' and '{word4}': {check_similarity(word1, word4, model)}")
print(f"Cosine similarity between '{word1}' and '{word5}': {check_similarity(word1, word5, model)}")
print(f"Cosine similarity between '{word1}' and '{word1}': {check_similarity(word1, word1, model)}")

Cosine similarity between 'cat' and 'dog': 0.9961859583854675
Cosine similarity between 'cat' and 'table': 0.9728946685791016
Cosine similarity between 'cat' and 'tank': 0.9930731654167175
Cosine similarity between 'cat' and 'cats': 0.998787522315979
Cosine similarity between 'cat' and 'cat': 1.0
